In [ ]:
using Pkg
Pkg.activate("Twitter")
Pkg.resolve()

In [ ]:
using Twitter, Graphs, GraphPlot, SparseArrays, LinearAlgebra, Colors, Plots

---
---
# Laden der Daten

Zunächst laden wir Daten von Twitter. 

dazu müssen wir uns zunächst authentifizieren.

Im Twitter [Developer Portal](https://developer.twitter.com/en) muss eine neue Application erstellt werden.

Dort können dann `CONSUMER_KEY`, `CONSUMER_SECRET`, `ACCESS_TOKEN` und `ACCESS_TOKEN_SECRET` generiert werden.

Siehe auch die [Dokumentation des Twitter.jl Packages](https://github.com/randyzwitch/Twitter.jl)



In [ ]:
CONSUMER_KEY = "";
CONSUMER_SECRET = "";
ACCESS_TOKEN = "";
ACCESS_TOKEN_SECRET = "";

twitterauth(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET);

Wir laden die letzten 200 Tweet, die das Wort `Osnabrück` enthalten.

In [ ]:
tweets = get_search_tweets(q = "Osnabrück", count = 200);

Wir wollen die Hashtags in diesen Tweets analysieren. 

Dazu erstellen wir eine Listen von Listen. Die $i$-te Liste enthält die Hashtags im $i$-ten Tweet

In [ ]:
T = tweets["statuses"]
hashtags = Vector{Vector{String}}()
for (i,t) in enumerate(T)
    push!(hashtags, Vector{String}())
    for entry in t.entities["hashtags"]
        push!(hashtags[i], entry["text"])
    end
end
filter!(x -> !isempty(x), hashtags)

Außerdem erstellen wir das Array `unique_hashtags`, welches alle Hashtags einzeln listet.

In [ ]:
all_hashtags = vcat(hashtags...)
unique_hashtags = unique(all_hashtags)

---
---
# Definition des Graphen

Aus den Daten erstellen wir ein Netzwerk.

Die Knoten im Graphen sind die Hashtags.

Wir definieren eine Kante zwischen zwei Knoten, falls die zwei Hashtags in einem gemeinsamen Tweet auftauchen.

Zuerst berechnen wir die Adjazenzmatrix.

In [ ]:
n = length(unique_hashtags)
A = zeros(Int, n, n)

function edge(hᵢ, hⱼ)
    for h in hashtags
        if hᵢ in h && hⱼ in h
            return 1
        end
    end
    return 0
end
    
    
for i in 2:(n-1)
    hᵢ = unique_hashtags[i]
    for j in (i+1):n
        hⱼ = unique_hashtags[j]
        A[i,j] = edge(hᵢ, hⱼ)
    end
end    
A = sparse(max.(A, A'))

die Adjazenzmatrix `A` definiert den Graphen `G`.

In [ ]:
G = Graphs.Graph(A)

---
---
# Visualisierung

Wir visualisieren den Graphen wie folgt.

Dabei plotten wir Labels nur für jene Knoten, die einene Grad größer als 3 haben.

In [ ]:
d = Graphs.degree(G)
c = findall(d .> 5)
labels = ["" for _ in 1:n]
labels[c] = unique_hashtags[c] 
layout= x -> spring_layout(x; C=25)
gplot(G, layout=layout, 
        linetype="curve", 
        nodelabel=labels,
        nodelabeldist=5,
        nodefillc = colorant"steelblue"
)

---
---
# Die Laplace Matrix

Wir berechnen die Laplace Matrix `L` von `G`.

In [ ]:
L = zeros(n, n)
for i in 1:n
    for j in 1:n
        if i == j
            L[i,j] = 1
        elseif A[i,j] == 1
            L[i,j] = - 1 / sqrt(d[i] * d[j])
        end
    end
end
L

die Eigenwerte von `L` werden dann wie folgt berechnet:

In [ ]:
E = eigen(L)
λ = E.values

In [ ]:
plot(λ, legend = false, linewidth = 3)

Die Summe der Eigenwerte ist gleich der Anzahl der Knoten.

In [ ]:
sum(λ), n

Ist `G` bipartit?

In [ ]:
is_bipartite(G)

Wir analysieren einen Eigenvektor zum größten Eigenwert. 

In [ ]:
v = E.vectors[:, n]
I = findall(abs.(v) .>  1e-8)
unique_hashtags[I]

Dieser eigenvektor definiert einen Untergraphen, der wie folgt aussieht:

In [ ]:
H = Graphs.Graph(A[I,I])

In [ ]:
labels = unique_hashtags[I]
layout= x -> spring_layout(x; C=21)
gplot(H, layout = layout,
        nodefillc = colorant"steelblue",
        linetype="curve", 
        nodelabel=labels,
        nodelabeldist=-1
)

Wir berechnen auch die Zusammenhangskomponenten

In [ ]:
C = connected_components(G)
filter!(c -> length(c)>1, C)

Wir plotten einer der Komponenten

In [ ]:
J = C[1]
K = Graphs.Graph(A[J,J])
d = Graphs.degree(K)
c = findall(d .> 3)
labels = ["" for _ in 1:length(J)]
labels[c] = unique_hashtags[J][c] 
layout= x -> spring_layout(x; C=21)
gplot(K, layout = layout,
        nodefillc = colorant"steelblue",
        linetype="curve", 
        nodelabel=labels,
        nodelabeldist=-2
)